In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [2]:
movies_data_path = "../data/movies.csv"
ratings_data_path = "../data/ratings.csv"
movies_1_data_path = "../data/movies1.csv"
ratings_1_data_path = "../data/ratings1.csv"
movie_matrix_path = "../data/movie_matrix.csv"
ratings_apprentissage_path = "../data/ratings_apprentissage.csv"

In [3]:

df_movies_1 = utils.init_traitement_DB(movies_1_data_path)
df_ratings_1 = utils.init_traitement_DB(ratings_1_data_path).drop(columns=['timestamp'])
df_movies1_trunc, df_ratings1_trunc = utils.filter_dataframes_by_threshold(2000, df_movies_1, df_ratings_1)

df_ratings_apprentissage, df_ratings_validation = train_test_split(df_ratings1_trunc, test_size=0.2, random_state=42)

#df_ratings_apprentissage = utils.init_traitement_DB(ratings_apprentissage_path).drop(columns=['timestamp'])

Reduction in movies: 16.68%
Reduction in ratings: 94.96%


In [4]:
df_movie_matrix = utils.create_db_content_movie(df_movies_1)
print("shape : ", df_movie_matrix.shape)
df_movie_matrix.head()

shape :  (57361, 19)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir
movieId,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
user_profiles = utils.calculate_user_profiles(df_ratings_apprentissage, df_movie_matrix)

Calculating user profiles...


In [6]:
user_profiles.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir
115211,0.047770,0.016009,0.014896,0.121137,0.025597,0.073367,0.287732,0.058471,0.070884,0.106926,0.028679,0.034843,0.035357,0.007020,0.017550,0.026624,0.009503,0.010273,0.007362
32456,0.054337,0.007701,0.019146,0.164723,0.029736,0.088245,0.258209,0.061825,0.064071,0.091026,0.023639,0.035512,0.023853,0.002032,0.010803,0.030164,0.016900,0.011552,0.006525
49000,0.060389,0.024685,0.029837,0.140527,0.041643,0.120349,0.245349,0.063752,0.037994,0.076059,0.023183,0.028334,0.042501,0.016171,0.006583,0.015455,0.017172,0.006297,0.003721
23763,0.054258,0.009797,0.019781,0.129427,0.036172,0.037773,0.164940,0.078372,0.047193,0.121703,0.137811,0.034194,0.075358,0.010268,0.018839,0.008572,0.007630,0.005463,0.002449
28812,0.073756,0.029293,0.024735,0.143551,0.030339,0.048872,0.175534,0.134210,0.082424,0.120012,0.014123,0.024735,0.042669,0.018458,0.006725,0.019504,0.003139,0.006277,0.001644


In [7]:
cluster_labels = utils.determine_optimal_clusters_para(user_profiles, cluster_method='K_means', max_clusters=10)

For n_clusters = 3, the average silhouette_score is : 0.20544463501428903
For n_clusters = 8, the average silhouette_score is : 0.17056272495358377
For n_clusters = 9, the average silhouette_score is : 0.16966247403597218
For n_clusters = 10, the average silhouette_score is : 0.16160887110700706
For n_clusters = 5, the average silhouette_score is : 0.17368220640198134
For n_clusters = 7, the average silhouette_score is : 0.1637472375914321
For n_clusters = 2, the average silhouette_score is : 0.28248519271410927
For n_clusters = 6, the average silhouette_score is : 0.16274466793354433
For n_clusters = 4, the average silhouette_score is : 0.2006982298094187
Optimal number of clusters is 2


In [ ]:
# Fonction pour prédire les évaluations et calculer l'erreur de prédiction
def predict(user_profiles, content_matrix, cluster_labels, df_ratings_validation, user_id_to_index):
    predictions = []
    true_ratings = []

    for index, row in df_ratings_validation.iterrows():
        user_id = row['userId']
        movie_id = row['movieId']
        true_rating = row['rating']

        user_profile = user_profiles.loc[user_id]
        user_cluster = cluster_labels[user_id_to_index[user_id]]  # Use the correct index
        cluster_users = user_profiles[cluster_labels == user_cluster]
        cluster_profile = cluster_users.mean()

        if movie_id in content_matrix.index:
            similarity_scores = content_matrix.loc[movie_id].dot(cluster_profile)
            # Adding influence of the average rating of the movie
            movie_ratings = df_ratings_apprentissage[df_ratings_apprentissage['movieId'] == movie_id]['rating']
            if not movie_ratings.empty:
                average_movie_rating = movie_ratings.mean()
                predicted_rating = (similarity_scores + average_movie_rating) / 2
        else:
            predicted_rating = cluster_profile.mean()  # Default prediction if movie not in content_matrix

        predictions.append(predicted_rating)
        true_ratings.append(true_rating)

    mse = mean_squared_error(true_ratings, predictions)
    print(f"Mean Squared Error: {mse}")
    return mse


In [9]:
user_profiles.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir
115211,0.047770,0.016009,0.014896,0.121137,0.025597,0.073367,0.287732,0.058471,0.070884,0.106926,0.028679,0.034843,0.035357,0.007020,0.017550,0.026624,0.009503,0.010273,0.007362
32456,0.054337,0.007701,0.019146,0.164723,0.029736,0.088245,0.258209,0.061825,0.064071,0.091026,0.023639,0.035512,0.023853,0.002032,0.010803,0.030164,0.016900,0.011552,0.006525
49000,0.060389,0.024685,0.029837,0.140527,0.041643,0.120349,0.245349,0.063752,0.037994,0.076059,0.023183,0.028334,0.042501,0.016171,0.006583,0.015455,0.017172,0.006297,0.003721
23763,0.054258,0.009797,0.019781,0.129427,0.036172,0.037773,0.164940,0.078372,0.047193,0.121703,0.137811,0.034194,0.075358,0.010268,0.018839,0.008572,0.007630,0.005463,0.002449
28812,0.073756,0.029293,0.024735,0.143551,0.030339,0.048872,0.175534,0.134210,0.082424,0.120012,0.014123,0.024735,0.042669,0.018458,0.006725,0.019504,0.003139,0.006277,0.001644


In [10]:
user_id_to_index = {user_id: index for index, user_id in enumerate(user_profiles.index)}



In [11]:
#optimal_clusters_silhouette = utils.determine_optimal_clusters_para(user_profiles, cluster_method='silhouette', max_clusters=10, n_jobs=-1)
#print(f"The optimal number of clusters for silhouette is: {optimal_clusters_silhouette}")

# Appliquer le clustering spectral
#cluster_labels_silhouette = utils.apply_spectral_clustering(user_profiles, optimal_clusters_silhouette)

#optimal_clusters_Kmeans = determine_optimal_clusters(user_profiles, cluster_method='K_means', max_clusters=10, n_jobs=-1)
optimal_clusters_Kmeans = utils.determine_optimal_clusters_para(user_profiles, cluster_method='K_means', max_clusters=5)
print(f"The optimal number of clusters for Kmeans is: {optimal_clusters_Kmeans}")

# Appliquer le clustering Kmeans
cluster_labels_Kmeans = utils.apply_kmeans_clustering(user_profiles, optimal_clusters_Kmeans)

#print("Shape of cluster_labels_silhouette: ", cluster_labels_silhouette.shape)
print("Shape of cluster_labels_Kmeans: ", cluster_labels_Kmeans.shape)


For n_clusters = 2, the average silhouette_score is : 0.28248519271410927
For n_clusters = 3, the average silhouette_score is : 0.20544463501428903
For n_clusters = 5, the average silhouette_score is : 0.17368220640198134
For n_clusters = 4, the average silhouette_score is : 0.2006982298094187
Optimal number of clusters is 2
The optimal number of clusters for Kmeans is: 2
Shape of cluster_labels_Kmeans:  (434,)


In [12]:
# Comparer les deux méthodes de clustering
#mse_spectral = predict(user_profiles, df_movie_matrix, cluster_labels_silhouette, df_ratings_validation, user_id_to_index)

#print(f"MSE for Spectral Clustering: {mse_spectral}")


In [13]:
mse_kmeans = predict(user_profiles, df_movie_matrix, cluster_labels_Kmeans, df_ratings_validation, user_id_to_index)

print(f"MSE for Kmeans Clustering: {mse_kmeans}")

Mean Squared Error: 2.758442272631711
MSE for Kmeans Clustering: 2.758442272631711


In [14]:
cluster_labels_Kmeans_200 = utils.apply_kmeans_clustering(user_profiles, 200)

mse_kmeans_200 = predict(user_profiles, df_movie_matrix, cluster_labels_Kmeans_200, df_ratings_validation, user_id_to_index)

print(f"MSE for Kmeans Clustering 200: {mse_kmeans_200}")

Mean Squared Error: 2.743807933091572
MSE for Kmeans Clustering 200: 2.743807933091572


In [15]:
        #user_profile = user_profiles.loc[user_id]
        #user_cluster = cluster_labels[user_id_to_index[user_id]] 
        #cluster_users = user_profiles[cluster_labels == user_cluster]
        #cluster_profile = cluster_users.mean()
#
        #movie_ratings = df_ratings_apprentissage[df_ratings_apprentissage['movieId'] == movie_id]['rating']
        #movie_ratings = movie_ratings[movie_ratings['userId'].isin(cluster_users.index)]